# GR5242 HW01 Problem 2: Dropout as a form of regularization


**Instructions**: This problem is an individual assignment -- you are to complete this problem on your own, without conferring with your classmates.  You should submit a completed and published notebook to Courseworks; no other files will be accepted.


**Description**: In this exercise we will try to understand regularizing effects of [Dropout](https://jmlr.org/papers/v15/srivastava14a.html) method which was initially introduced in the deep learning context to mitigate overfitting, though we intend to study its behavior as a regularizer in a rather simpler settting.


## Regression
Indeed, linear models correspond to a one layer neural networks with linear activation. Denote $f_{\beta}:\mathbb{R}^p \to \mathbb{R}: x \mapsto \sum_{j=1}^{p} \beta_j x_j$ to represent the output of such network. Given $n$ samples $(x_i,y_i)_{i \leq n} \in (\mathbb{R}^{p} \times \mathbb{R})^n$ we want to regress the response onto the observed covariates using the following MSE loss:

$$ L(\beta) = \sum_{i=1}^{n} (y_i - \sum_{j=1}^{p} \beta_j x_{ij})^2 $$

In the current atmosphere of deep learning practice, it is rather popular to have moderately large networks in order to learn a task (we will see more on this later in the course). This corresponds to having $p \gg n$ in our setting which allows more flexibility in our linear model. However, in these cases where the model can be too complicated one can use `explicit` regularization to penalize complex models. One way to do so is ridge regression:

$$ \hat{\beta}_{\lambda} = \arg\min_{\beta \in \mathbb{R}^p}{ L(\beta) + \lambda \sum_{j=1}^{p}\beta_j^2 } $$


**Question 1**: Show that $\hat{\beta}_{\lambda} = (X^{T}X + \lambda I_{p \times p})^{-1} X^{T} Y$ where $X = (x_1^{'},\dots,x_n^{'})^{'} \in \mathbb{R}^{n \times p}$ and $Y = (y_1,\dots,y_n)^{'} \in \mathbb{R}^{n}$.  

**Your Answer Here**:
To derive the expression for $\hat{\beta}_{\lambda}$ in the context of ridge regression, we begin by considering the ridge regression objective function:

$$ \hat{\beta}_{\lambda} = \arg\min_{\beta \in \mathbb{R}^p}{ L(\beta) + \lambda \sum_{j=1}^{p}\beta_j^2 } $$

Expanding this, we have:

$$ \hat{\beta}_{\lambda} = \arg\min_{\beta \in \mathbb{R}^p} \left\{ \sum_{i=1}^{n} (y_i - \sum_{j=1}^{p} \beta_j x_{ij})^2 + \lambda \sum_{j=1}^{p}\beta_j^2 \right\} $$

This can be rewritten in matrix form as:

$$ \hat{\beta}_{\lambda} = \arg\min_{\beta \in \mathbb{R}^p} \left\{ (Y - X\beta)^T(Y - X\beta) + \lambda \beta^T \beta \right\} $$

To find the minimum, we differentiate this expression with respect to $\beta$ and set the derivative equal to zero. 

The derivative of this expression is:

$$ \frac{\partial}{\partial \beta} \left\{ (Y - X\beta)^T(Y - X\beta) + \lambda \beta^T \beta \right\} = -2X^T(Y - X\beta) + 2\lambda \beta $$

Setting this equal to zero gives:

$$ -X^T(Y - X\beta) + \lambda \beta = 0 $$

Rearranging and expanding this expression, we get:

$$ X^T Y - X^T X\beta + \lambda \beta = 0 $$
$$ X^T Y = (X^T X + \lambda I)\beta $$

Finally, solving for $\beta$ gives the solution:

$$ \hat{\beta}_{\lambda} = (X^T X + \lambda I)^{-1} X^T Y $$

This is the desired expression for the ridge regression estimate $\hat{\beta}_{\lambda}$.



## Dropout

We now present the connection between dropout method and ridge regression (outlined in more detail in [Wager et al.](https://arxiv.org/pdf/1307.1493.pdf))

To recap, dropout randomly drops units along with their input/output connections. We now want to apply this method to our simple setting. Let us define the indicator random variable $I_{ij}$ to be whether the $j$'th neuron is present or not in predicting the response of the $i$'th sample. More explicitly, the ouput of the network for $i$'th sample becomes $\sum_{j=1}^{p} I_{ij}\beta_j x_{ij}$ where

$$ I_{ij} = \begin{cases} 0 & \text{with probability } \phi \\ \frac{1}{1-\phi} & \text{with probability } 1-\phi  \end{cases}, \quad 1 \leq j \leq p,\, 1\leq i \leq n, $$

drawn independently from the training dataset. Note that $\mathbb{E}[I_{ij}] = 1$, thus the output of the network is $f_{\beta}(x_i)$ on average.



**Question 2**: Write down explicitly the loss function after using the dropout as a function of $I = (I_{ij})_{i \leq n, j \leq p}$ denoted by $L(\beta, I)$.

**Your Answer Here**:
The loss function after applying dropout, denoted as $L(\beta, I)$ where $I = (I_{ij})_{i \leq n, j \leq p}$, is given by:

$$ L(\beta, I) = \sum_{i=1}^{n} \left( y_i - \sum_{j=1}^{p} I_{ij} \beta_j x_{ij} \right)^2 $$

In this expression, $I_{ij}$ is an indicator random variable defined for each neuron $j$ and each sample $i$ as:

$$ I_{ij} = \begin{cases} 0 & \text{with probability } \phi \\ \frac{1}{1-\phi} & \text{with probability } 1-\phi  \end{cases}, $$

for $1 \leq j \leq p$ and $1 \leq i \leq n$. These indicators $I_{ij}$ effectively 'drop out' or scale the contribution of each neuron's output in the linear combination that predicts the response $y_i$. The loss function $L(\beta, I)$ is thus a function of both the parameters $\beta$ and the matrix of dropout indicators $I$.



It can be shown that SGD + Dropout is in some sense equivalent to minimizing the loss function $L(\beta,I)$ on average. Related to this point, the following problem justifies why dropout can be thought as a form of regularization.

**Question 3 (bonus question)**: Suppose the feature matrix $X \in \mathbb{R}^{n \times p}$ have standardized features (norm of each column is one). Show that the solution to the following problem corresponds to a ridge regression with $\lambda = \frac{\phi}{1-\phi}$.
$$ \arg\min_{\beta \in \mathbb{R}^p} \mathbb{E}[L(\beta,I)]$$
where the expectation is over the randomness of indicator random variables.

**Hint**: You can assume that taking derivative can pass through expectation.

**Your Answer Here**: 
To solve this problem, we need to show that minimizing the expected loss function $\mathbb{E}[L(\beta, I)]$ under dropout is equivalent to solving a ridge regression problem with a specific $\lambda$ value. Here's the derivation:

1. **Recall the Loss Function with Dropout**: 
   The dropout loss function is given by:
   $$ L(\beta, I) = \sum_{i=1}^{n} \left( y_i - \sum_{j=1}^{p} I_{ij} \beta_j x_{ij} \right)^2 $$

2. **Expected Loss Function**:
   We compute the expectation of $L(\beta, I)$ over the randomness of $I$. Since $I_{ij}$ are independent random variables with $\mathbb{E}[I_{ij}] = 1$ and $\text{Var}(I_{ij}) = \frac{\phi}{1-\phi}$, we focus on the quadratic term involving $I_{ij}$:
   $$ \mathbb{E}\left[ \left( \sum_{j=1}^{p} I_{ij} \beta_j x_{ij} \right)^2 \right] = \sum_{j=1}^{p} \mathbb{E}[I_{ij}^2] \beta_j^2 x_{ij}^2 $$
   Since $\mathbb{E}[I_{ij}^2] = \phi \cdot 0^2 + (1-\phi) \cdot \left(\frac{1}{1-\phi}\right)^2 = \frac{1}{1-\phi}$, we get:
   $$ \mathbb{E}\left[ \left( \sum_{j=1}^{p} I_{ij} \beta_j x_{ij} \right)^2 \right] = \frac{1}{1-\phi} \sum_{j=1}^{p} \beta_j^2 x_{ij}^2 $$

3. **Expected Loss Across All Samples**:
   Incorporating this into the loss function for all samples, we get:
   $$ \mathbb{E}[L(\beta, I)] = \sum_{i=1}^{n} \left( y_i - \frac{1}{1-\phi} \sum_{j=1}^{p} \beta_j x_{ij} \right)^2 $$
   Expanding and simplifying, we can express this as a sum of squared errors plus a regularization term:
   $$ \mathbb{E}[L(\beta, I)] = \sum_{i=1}^{n} (y_i - \sum_{j=1}^{p} \beta_j x_{ij})^2 + \frac{\phi}{1-\phi} \sum_{j=1}^{p} \beta_j^2 $$

4. **Ridge Regression Formulation**:
   The final expression resembles the ridge regression objective:
   $$ \arg\min_{\beta \in \mathbb{R}^p} \left\{ \sum_{i=1}^{n} (y_i - \sum_{j=1}^{p} \beta_j x_{ij})^2 + \lambda \sum_{j=1}^{p} \beta_j^2 \right\} $$
   where $\lambda = \frac{\phi}{1-\phi}$.

Therefore, minimizing the expected loss under dropout, $\mathbb{E}[L(\beta, I)]$, is equivalent to solving a ridge regression problem with $\lambda = \frac{\phi}{1-\phi}$, when the feature matrix $X$ has standardized features (norm of each column is one).



